In [0]:
catalog_name = "dmc_tallerfinaldatabricks_mariovento"

schema_bronze = "bronze"
schema_silver = "silver"

In [0]:
from pyspark.sql.types import DecimalType, StringType, LongType
from pyspark.sql.functions import col, when, concat, lit, trim, initcap, expr

In [0]:
silver_productos = (
    spark.table(f"{catalog_name}.{schema_bronze}.productos_raw")
    # Convierte id_producto a bigint
    .withColumn("id_producto", col("id_producto").cast("bigint"))
    # Limpia y estandariza el nombre del producto
    .withColumn(
        "nombre",
        when(
            col("nombre").isNull(), 
            concat(lit("Producto"), lit(" "), expr("cast(id_producto as string)"))
        )
        .when(
            col("nombre").rlike("(?i).*producto.*"), 
            initcap(trim(col("nombre")))
        )
        .otherwise(
            concat(initcap(trim(col("nombre"))), lit(" "), expr("cast(id_producto as string)"))
        )
    )
    # Limpia y estandariza la categoría
    .withColumn(
        "categoria", 
        initcap(trim(col("categoria"))).cast("string")
    )
    # Valida y corrige el precio, asignando 0 si es negativo
    .withColumn(
        "precio", 
        when(
            expr("try_cast(precio as decimal(18,2))") < 0, 
            lit(0)
        ).otherwise(
            expr("try_cast(precio as decimal(18,2))")
        )
    )
    .withColumn("precio_corregido", when(col("precio") < 0, 0).otherwise(col("precio")))
    .withColumn("estado_precio", when(col("precio") < 0, "Corregido").otherwise("No corregido"))
    # Asigna "Desconocida" si la categoría es nula
    .fillna("Desconocida", subset=["categoria"])
    # Elimina duplicados por id_producto
    .dropDuplicates(["id_producto"])
)

In [0]:
display(silver_productos)

In [0]:
silver_productos.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_silver}.productos")

In [0]:
%sql
select distinct categoria 
from dmc_tallerfinaldatabricks_mariovento.silver.productos